## 扱う質問

「神戸情報大学院大学」に関する質問を取り扱います。これは以下の背景を持つトピックです。

- 令和４年４月１日から施行されている神戸情報大学院大学の学則
- 2025年4月1日に改定された神戸情報大学院大学のデジタルパンフレット
- 上記２文書のPDFファイルをyomitokuによりマークダウン化した物

## 扱うモデル

~~「Qwen/Qwen3-8B-FP8」を使用します。このモデルは、以下の特徴を持ちます。~~

- 最近出た凄いって言われてるモデルを使ってみたいって感じです。
  
「Qwen/Qwen3-8B-AWQ」を使用します。このモデルは、以下の特徴を持ちます。

- 最近出た凄いって言われてるモデルであるQwen3-8BのAWQ版です。
- transformersライブラリにより非推奨ながらサポートされ使用メモリ削減が期待できます。

### 演習環境の準備

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes
# または特定のバージョンを指定する場合
# pip install "autoawq<0.2.7"

In [ ]:
!pip install autoawq

In [8]:
!pip show transformers
# バージョンが古い場合は
#pip install --upgrade transformers

Name: transformers
Version: 4.51.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [2]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [1]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import random
random.seed(0)

In [3]:

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch # torch_dtype を使う場合、あるいは device 設定などで必要になることがある

# 使用するデバイスを確認（AWQモデルもGPUで動かすことが前提）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# AWQ量子化済みモデル名
# 注意: "Qwen/Qwen3-8B-AWQ" というモデルがHugging Face Hubに実際に存在するか確認してください。
# もし存在しない場合は、適切なAWQモデル名に置き換えるか、
# Qwen3-8BのFP16版などをAutoAWQで自分で量子化する必要があります。
model_name = "Qwen/Qwen3-8B-AWQ"

# YaRN設定 (例: factor=4.0 で 32768 * 4.0 = 131072 トークンを目指す)
# original_max_position_embeddings はモデルのネイティブコンテキスト長
rope_scaling_config = {"type": "yarn", "factor": 4.0, "original_max_position_embeddings": 32768}

# トークナイザーのロード (モデルロードの前にやっても後でもOK)
# trust_remote_code はトークナイザーにも必要な場合がある
try:
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True # モデルによっては必要
    )
    print(f"Tokenizer for '{model_name}' loaded successfully.")
except Exception as e:
    print(f"Error loading tokenizer for '{model_name}': {e}")
    tokenizer = None

# AWQ量子化済みモデルのロード
# AWQモデルの場合、quantization_config や torch_dtype は通常不要
if tokenizer is not None: # トークナイザーが正常にロードされた場合のみモデルをロード
    try:
        print(f"Loading AWQ model '{model_name}'...")
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",  # 利用可能なデバイスに自動配置 (通常はGPU)
            trust_remote_code=True, # Qwenモデルでは必要になることが多い
            rope_scaling=rope_scaling_config, # YaRN設定
            # attn_implementation="flash_attention_2", # もしFlashAttention2が使え、モデルがサポートしていればパフォーマンス向上
        )
        print(f"Model '{model_name}' loaded successfully. Device: {model.device}")

        # モデルがGPUにロードされたか確認
        if model.device.type != 'cuda' and torch.cuda.is_available():
            print(f"Warning: Model might not be on GPU. model.device is {model.device}")

    except Exception as e:
        print(f"Error loading AWQ model '{model_name}': {e}")
        print("Possible reasons: Model not found on Hugging Face Hub, VRAM insufficiency, or autoawq library not installed/configured correctly.")
        print("Please ensure 'autoawq' is installed (pip install autoawq) if you are loading AWQ models with transformers.")
        model = None
else:
    print("Tokenizer loading failed, skipping model loading.")
    model = None

# これ以降、modelとtokenizerを使って推論処理などを行う
# if model and tokenizer:
#     # 推論コードの例
#     prompt = "こんにちは、Qwen3！"
#     messages = [{"role": "user", "content": prompt}]
#     text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
#     generated_ids = model.generate(**model_inputs, max_new_tokens=50)
#     response = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[-1]:], skip_special_tokens=True)
#     print("Response:", response)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Tokenizer for 'Qwen/Qwen3-8B-AWQ' loaded successfully.
Loading AWQ model 'Qwen/Qwen3-8B-AWQ'...


/usr/local/lib/python3.11/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model 'Qwen/Qwen3-8B-AWQ' loaded successfully. Device: cuda:0


In [4]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available. Running on CPU.")

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
Number of GPUs: 1
Current CUDA device: 0
Device name: NVIDIA GeForce RTX 4080


# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [9]:
def generate_output_qwen3(query, model, tokenizer, enable_thinking=True, max_new_tokens=1024):
    """
    Qwen3モデル用に調整されたテキスト生成関数。デフォルトで日本語応答を促すシステムプロンプトを含む。

    Args:
        query (str): モデルへの質問文。
        model: ロード済みのQwen3モデル。
        tokenizer: Qwen3モデルに対応するトークナイザー。
        enable_thinking (bool): Qwen3のThinking Modeを有効にするか。Trueで有効。
        max_new_tokens (int): 生成する最大トークン数。

    Returns:
        str: モデルからの応答テキスト（Thinking Modeの場合は最終回答部分）。
    """
    # messages リストの先頭に、日本語での応答を促すシステムプロンプトを追加
    messages = [
        {"role": "system", "content": "回答は日本語で行ってください。"}, # 日本語応答を促すシステムプロンプト
        {"role": "user", "content": query},
    ]

    # 1. チャットテンプレートの適用 (enable_thinking を追加)
    # tokenizer.apply_chat_template は messages リストを直接処理します。
    # Qwenモデルの場合、systemプロンプトはチャットテンプレートによって適切にフォーマットされます。
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        enable_thinking=enable_thinking # Qwen3特有の引数
    ).to(model.device)

    # 2. 終了トークンの設定
    eos_token_id_to_use = tokenizer.eos_token_id
    if eos_token_id_to_use is None and hasattr(tokenizer, 'eod_id'): # Qwen特有の属性
        eos_token_id_to_use = tokenizer.eod_id


    # 3. サンプリングパラメータの設定
    generation_kwargs = {
        "max_new_tokens": max_new_tokens,
        "eos_token_id": eos_token_id_to_use,
        "pad_token_id": eos_token_id_to_use, # warning: `pad_token_id` is not set... を避ける
    }

    if enable_thinking:
        generation_kwargs.update({
            "do_sample": True,
            "temperature": 0.6,
            "top_p": 0.95,
            "top_k": 20,
        })
    else:
        generation_kwargs.update({
            "do_sample": True,
            "temperature": 0.7,
            "top_p": 0.8,
            "top_k": 20,
        })
    
    # 4. テキスト生成
    outputs = model.generate(
        input_ids,
        **generation_kwargs
    )

    # 5. 応答部分の抽出とデコード
    response_ids = outputs[0][input_ids.shape[-1]:]

    if enable_thinking:
        full_response_decoded = tokenizer.decode(response_ids, skip_special_tokens=False) # 特殊トークンを含めてデコード
        
        think_end_tag = "</think>"
        final_answer = ""
        
        think_end_idx = full_response_decoded.rfind(think_end_tag)
        if think_end_idx != -1:
            raw_final_answer = full_response_decoded[think_end_idx + len(think_end_tag):]
            final_answer = tokenizer.decode(tokenizer.encode(raw_final_answer, add_special_tokens=False), skip_special_tokens=True).strip()
        else:
            final_answer = tokenizer.decode(response_ids, skip_special_tokens=True).strip()
        
        return final_answer
    else:
        return tokenizer.decode(response_ids, skip_special_tokens=True).strip()

### ベースライン

In [10]:
question =  "神戸情報大学院大学の現学長の名前と主なプロフィール（経歴や役職）を3点挙げてください。"
# Thinking Mode を有効にして生成
response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
print(f"Thinking ON: {response_on}")


Thinking ON: 神戸情報大学院大学（こうべじょうほうだいがくいんだい）の現学長は**荒木 健一氏（あらき けんいち）**です。以下に主なプロフィールを3点挙げます：

1. **学術的背景と専門分野**  
   荒木学長は、情報工学・コンピュータサイエンス分野に特化した研究者として知られています。東京大学大学院工学系研究科を修了し、博士号（工学）を取得。特に、人工知能、データベース、ネットワーク技術などの分野で幅広い研究経験を持ち、学術的な実績が認められています。

2. **大学の行政経験**  
   神戸情報大学院大学では、学長補佐や学長としての役職を歴任し、大学の教育・研究体制の整備に貢献。また、大学の国際化戦略や産業界との連携強化に注力し、実学的教育の推進を主導してきました。

3. **教育・研究への貢献**  
   学長として、学際的な教育プログラムの開発や、研究インフラの拡充に取り組んでいます。特に、社会のニーズに対応した実践的な研究プロジェクトの推進や、学生の実践力向上を目的としたカリキュラムの改革を進めています。

（※ 2024年7月現在の情報に基づいています。最新の変更がある場合は、公式ウェブサイトでご確認ください。）


In [11]:
question =  "神戸情報大学院大学の現学長の名前と主なプロフィール（経歴や役職）を3点挙げてください。"

# Thinking Mode を無効にして生成
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking OFF: 神戸情報大学院大学の現学長は**小林正明氏**です。以下に、小林学長の主なプロフィールと経歴を3点挙げます。

1. **学歴**：東京大学大学院工学系研究科博士課程修了（情報理工学専攻）。  
2. **経歴**：東京大学大学院情報理工学系研究科教授、東京大学大学院情報学研究科教授などを歴任。  
3. **現職**：神戸情報大学院大学学長（2022年4月から就任）。  

小林学長は、情報工学、コンピュータビジョン、画像処理、機械学習などの分野で幅広い研究経験を持ち、教育・研究の分野で活躍されています。


In [12]:
question =  "神戸情報大学院大学に入学する場合、学則第30条および別表1に定められている入学金と年間の授業料はそれぞれいくらですか？"
# response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
# print(f"Thinking ON: {response_on}")
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking OFF: 神戸情報大学院大学の学則第30条および別表1に定められている入学金と年間の授業料について、正確な金額は毎年変更される可能性があるため、最新の情報を確認する必要があります。

一般的に、日本の大学院（大学院大学）の入学金と年間授業料は以下のようになります（2024年度の予定値を参考にしたものです）：

- **入学金**：約 **100,000円**（例：100,000円）
- **年間授業料**：約 **1,500,000円〜2,000,000円**（例：1,800,000円）

ただし、上記の金額は、学部・大学院の種類やコースによって異なり、また、教育内容や施設利用、研究活動などに応じて変動することがあります。

### 確認方法：
1. **神戸情報大学院大学の公式ウェブサイト**  
   [公式サイト](https://www.kobe-uni.ac.jp/)にアクセスし、「学則」や「学費」に関するページを確認してください。

2. **入試案内やパンフレット**  
   入試情報や入学者向けの資料に記載されています。

3. **学務課や総務課**  
   学生相談窓口や総務課に直接お問い合わせすると、最新の金額や詳細な説明を受けられます。

ご希望であれば、公式サイトのリンクや具体的なページの情報をご案内することも可能です。お気軽にお知らせください。


In [13]:
question =  "神戸情報大学院大学において「フルオンライン履修制度」を利用して正規課程を修了した場合、学則に基づき授与される学位の正式名称は何ですか？"
# response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
# print(f"Thinking ON: {response_on}")
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking OFF: 神戸情報大学院大学において、「フルオンライン履修制度」を利用して正規課程を修了した場合、学則に基づき授与される学位の正式名称は、

**「修士（情報学）」** または **「博士（情報学）」** となります。

ただし、学位の種別（修士・博士）は、コース（修士課程／博士課程）によって異なります。  
例えば：

- **修士課程**：修士（情報学）
- **博士課程**：博士（情報学）

また、学位の名称は、大学の学則や学位授与に関する規定に従って決定されるため、詳細は神戸情報大学院大学の公式サイトや学則等を確認することをお勧めします。


In [14]:
question = "神戸情報大学院大学では「100カ国を超える国々からの留学生」を受け入れていることが特徴の一つです。この大学の特徴は、学則に定められる入学資格のどの項目によって具体的に支えられていると考えられますか？関連する項目を複数挙げ、その理由を説明してください。"
# response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
# print(f"Thinking ON: {response_on}")
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking OFF: 神戸情報大学院大学が「100カ国を超える国々からの留学生」を受け入れているという特徴は、その大学が国際的な教育環境を提供し、多文化・多国籍の学習環境を整えていることを示しています。この特徴は、大学の**学則に定められた入学資格の項目**によって具体的に支えられています。以下に、関連する入学資格の項目を複数挙げ、その理由を説明します。

---

### 1. **国際的な経験や語学能力の要件**

**関連項目：**  
- 外国語能力の証明（例：TOEFL、TEF、JLPTなど）  
- 外国での生活経験や国際的な活動の経歴  

**理由：**  
神戸情報大学院大学は国際的な教育を重視しており、留学生の受け入れに際して、**英語や日本語などの言語能力**や**国際的な経験**を評価しています。これにより、留学生が日本の教育環境や学習内容を十分に理解し、多文化・多国籍の学習環境に適応できるよう支援することが可能になります。また、国際的な経験がある学生は、国際的な視野を持ち、多様な価値観を受け入れる姿勢を持ちやすいとされています。

---

### 2. **学術的背景や専門知識の要件**

**関連項目：**  
- 学歴や学位の証明  
- 関連分野の知識や経験の証明  

**理由：**  
大学が留学生を受け入れる際には、**学術的背景や専門知識のレベル**を確認することが一般的です。これは、留学生が大学の教育内容や授業に適切に参加できるようにするためのものです。特に、神戸情報大学院大学は情報学やコンピュータ科学などの専門分野に特化しており、留学生もこれらの分野に興味を持ち、専門知識を活かせることが求められます。そのため、学歴や専門知識の要件は、留学生が大学の教育目標に合致していることを確認するための重要な要素です。

---

### 3. **国籍や地域の多様性への配慮**

**関連項目：**  
- 国籍の記載  
- 国際交流活動への参加歴  

**理由：**  
神戸情報大学院大学は「100カ国を超える国々からの留学生」を受け入れるという特徴を持っています。これは、**多国籍の学習環境を実現するための取り組み**の一環です。そのため、入学資格に「国籍」の記載や「国際交流活動への参加歴」などの項目が設けられていると

In [15]:
question =  "神戸情報大学院大学に在籍する高原敏竜特任教授は、国際機関のプロジェクト調整員や企画調査員として複数の国での活動経験があります。これらの国々の一般的な開発課題をいくつか挙げてください。その上で、高原特任教授が専門とする分野が、これらの開発課題の解決にどのように貢献できる可能性があるか、具体例を交えて説明してください。"
# response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
# print(f"Thinking ON: {response_on}")
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking OFF: 高原敏竜特任教授が所属する神戸情報大学院大学は、国際的な視点と地域社会への貢献を重視した教育研究機関です。高原特任教授は、国際機関のプロジェクト調整員や企画調査員として、複数の国での活動経験を持つことから、国際開発や地域社会の課題解決に深く関わる専門家として知られています。

### 1. 国々の一般的な開発課題の例

国際的な開発活動において、多くの国が直面している一般的な開発課題は以下の通りです：

- **教育・人材育成の格差**：特に発展途上国では、質の高い教育や専門的なスキルの習得が困難な場合があります。
- **環境問題と持続可能な開発**：気候変動、資源の枯渇、汚染など、環境への悪影響が深刻化しています。
- **医療・健康へのアクセスの不平等**：医療インフラの整備が不十分な地域では、感染症や慢性疾患への対応が困難です。
- **経済的貧困と雇用の不確実性**：失業や貧困が続く地域では、人々の生活の質が低下しています。
- **情報通信インフラの整備不足**：デジタルデバイドが深刻な地域では、情報の取得や教育・ビジネスの効率化に課題があります。

---

### 2. 高原特任教授の専門分野と開発課題への貢献

高原特任教授は、**国際開発・地域社会の支援、情報通信技術（ICT）の活用、国際プロジェクトの調整・企画**などを専門としています。これらの専門分野は、上述の開発課題の解決に直接的に貢献する可能性があります。

#### 具体例：**情報通信技術を活用した教育支援プロジェクト**

高原特任教授は、ICTを活用した教育支援プロジェクトに携わった経験があります。例えば、東南アジアの一部の国では、都市部と地方の教育格差が深刻です。高原特任教授は、このような地域において、**オンライン教育プラットフォームの構築や、ICTを活用した教育支援ツールの開発**に貢献しました。

- **貢献内容**：
  - リモート学習のためのデジタル教材の開発
  - 地方の学校にインターネットアクセスを提供するプロジェクトの設計
  - 教師向けのICT活用トレーニングの実施

- **成果**：
  - 地方の学校でも質の高い教育を受けられるようになる
  - 教師がICTを活用して授業をより効果的に行えるようになる
  - 学生の

## 結果 (ベースモデル)

「Qwen/Qwen3-8B-FP」は「神戸情報大学院大学」について誤った知識を提示しました：
* モデルは神戸情報大学院大学自体に対するデータを所持していないようで、完全に事実と異なる説明になります。

---

## 2.1 パンフレット及び学則をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: 公式パンフレット及び学則
* **目的**: モデルに「神戸情報大学院大学」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す


In [ ]:
# チャンク化のために LangChain の Text Splitters をインストール
!pip install langchain-text-splitters

### phytonで整理

In [11]:
import re
import os
# langchain_text_splitters はチャンク化の際に使うので、ここではインポートは不要
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# --- 前処理関数の定義 (前回の回答と同様) ---

def preprocess_digipanp_text(text):
    """
    digiPanpファイルのテキストを整形します。
    - <br> タグを改行に置換
    - 画像タグを除去
    - 単独の '0' を除去
    - 過剰な空白や改行を整理
    """
    cleaned_text = re.sub(r'<br\s*/?>', '\n', text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r'<img\s+[^>]*>', '', cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r'\n\n0\n\n', '\n\n', cleaned_text) # 連続改行に囲まれた単独0
    cleaned_text = re.sub(r'\n\s*\n', '\n\n', cleaned_text) # 過剰な空白行を1つにまとめる
    cleaned_text = cleaned_text.strip()
    return cleaned_text

# --- 分割関数の定義 (前回の回答と同様) ---
# ここでは分割はせず、整形後のテキストをファイル保存するだけなので、これらの関数は直接は使いませんが、定義は必要かもしれません。
# split_by_chapter は学則用
def split_by_chapter(text):
    split_pattern = r"(?m)^\s*(?=第\d+章)"
    chapters_raw = re.split(split_pattern, text)
    documents = [chapter.strip() for chapter in chapters_raw if chapter.strip()]
    return documents

# split_by_digipanp_page は digiPanp用
def split_by_digipanp_page(text):
    page_header_pattern = r"^\s*(?:\d{2}\s+Kobe Instutute ofc\.omputing|Kobe Institute of Computing\s+\d{2}).*$"
    split_pattern = r"(?m)(?=" + page_header_pattern + r")"
    pages_raw = re.split(split_pattern, text)
    documents = [page.strip() for page in pages_raw if page.strip()]
    return documents


# --- ファイルパスの設定 ---
gakusoku_file_path = "AI-E-03/RAG/kic-gakusoku.md"
digipanp_file_path = "AI-E-03/RAG/KIC_digiPanp.md"

# 整形後のファイルを保存するパス
cleaned_gakusoku_path = "AI-E-03/RAG/kic-gakusoku_cleaned.md"
cleaned_digipanp_path = "AI-E-03/RAG/KIC_digiPanp_cleaned.md"


# --- 学則ファイルを処理・整形・保存 ---
if os.path.exists(gakusoku_file_path):
    print(f"'{gakusoku_file_path}' を読み込み中...")
    with open(gakusoku_file_path, "r", encoding="utf-8") as f:
        raw_writedown_gakusoku = f.read()

    # 学則ファイルの前処理 (digipanpほどではないが、改行タグや余分な空白の整理など)
    print("学則ファイルに前処理を適用中...")
    cleaned_gakusoku_text = re.sub(r'<br\s*/?>', '\n', raw_writedown_gakusoku, flags=re.IGNORECASE)
    cleaned_gakusoku_text = re.sub(r'\n\s*\n', '\n\n', cleaned_gakusoku_text) # 過剰な空白行を1つにまとめる
    cleaned_gakusoku_text = cleaned_gakusoku_text.strip()

    # 整形後のファイルを保存
    print(f"整形後の学則ファイルを '{cleaned_gakusoku_path}' に保存中...")
    with open(cleaned_gakusoku_path, "w", encoding="utf-8") as f:
        f.write(cleaned_gakusoku_text)
    print("保存完了。")

else:
    print(f"エラー: '{gakusoku_file_path}' が見つかりません。パスを確認してください。")


# --- digiPanpファイルを処理・整形・保存 ---
if os.path.exists(digipanp_file_path):
    print(f"\n'{digipanp_file_path}' を読み込み中...")
    with open(digipanp_file_path, "r", encoding="utf-8") as f:
        raw_writedown_digipanp = f.read()

    # digiPanpファイルに前処理を適用
    print("digiPanpファイルに前処理を適用中...")
    preprocessed_digipanp_text = preprocess_digipanp_text(raw_writedown_digipanp) # digipanp向けの前処理関数を使用

    # 分割後クリーニングで使っていたフッター除去を、ファイル全体の前処理として適用する
    # これにより、ファイル全体からフッター行が削除される
    page_footer_pattern = r"(?m)^\s*(?:\d{2}\s+Kobe Instutute ofc\.omputing|Kobe Institute of Computing\s+\d{2}).*?$"
    cleaned_digipanp_text = re.sub(page_footer_pattern, '', preprocessed_digipanp_text, flags=re.IGNORECASE|re.MULTILINE)
    cleaned_digipanp_text = cleaned_digipanp_text.strip() # 再度トリム

    print("前処理完了。")

    # 整形後のファイルを保存
    print(f"整形後のdigiPanpファイルを '{cleaned_digipanp_path}' に保存中...")
    with open(cleaned_digipanp_path, "w", encoding="utf-8") as f:
        f.write(cleaned_digipanp_text)
    print("保存完了。")

else:
    print(f"\nエラー: '{digipanp_file_path}' が見つかりません。パスを確認してください。")

print("\n--- 整形完了 ---")

# 次のステップは、これらの整形済みファイルを読み込んで、
# RecursiveCharacterTextSplitter でチャンク化することです。
# その際には、split_by_chapter や split_by_digipanp_page のロジックで使った
# 章やページのパターンを separators として利用します。
# (整形済みファイルではこれらのパターン自体は残っている必要があります)

'AI-E-03/RAG/kic-gakusoku.md' を読み込み中...
学則ファイルに前処理を適用中...
整形後の学則ファイルを 'AI-E-03/RAG/kic-gakusoku_cleaned.md' に保存中...
保存完了。

'AI-E-03/RAG/KIC_digiPanp.md' を読み込み中...
digiPanpファイルに前処理を適用中...
前処理完了。
整形後のdigiPanpファイルを 'AI-E-03/RAG/KIC_digiPanp_cleaned.md' に保存中...
保存完了。

--- 整形完了 ---


### Gemini APIによる分割準備

In [17]:
!pip install google-generativeai

#### geminiで処理

In [4]:
import google.generativeai as genai
import os
import re
import time
from dotenv import load_dotenv

# .env ファイルからAPIキーを読み込む
# 相対パスと絶対パスの両方で試行
try:
    # まず相対パスで試行
    load_dotenv(dotenv_path="AI-E-03/.env", verbose=True)
    # 絶対パスでも試行（必要に応じて）
    if not os.getenv("GOOGLE_API_KEY"):
        absolute_path = os.path.join(os.getcwd(), "AI-E-03", ".env")
        load_dotenv(dotenv_path=absolute_path, verbose=True)

    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
except Exception as e:
    print(f"警告: .env ファイルの読み込み中にエラーが発生しました: {e}")
    GOOGLE_API_KEY = None

# APIキーが設定されているか確認
if not GOOGLE_API_KEY:
    print("エラー: 環境変数 'GOOGLE_API_KEY' が設定されていません。")
    exit()

# Gemini API の設定
api_configured = False
try:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Gemini API configured successfully.")
    api_configured = True
except Exception as e:
    print(f"エラー: Gemini API の設定に失敗しました: {e}")
    exit()

# 使用するGeminiモデル (最新版を指定)
MODEL_NAME = "gemini-2.5-flash-preview-04-17"

# チャンク化マーカー記号の定義
CHUNK_SEPARATOR_MARKER = "[CHUNK_SEPARATOR]"


def process_file_with_gemini(file_path, output_path, model_name, marker=CHUNK_SEPARATOR_MARKER):
    """
    Gemini APIを使ってファイル内容にチャンク区切りマーカーを挿入し、保存します。
    """
    # API設定が成功しているかチェック
    if not api_configured:
        print("エラー: Gemini API 設定が完了していません。処理をスキップします。")
        return

    # ファイルの存在チェック
    if not os.path.exists(file_path):
        print(f"エラー: ファイル '{file_path}' が見つかりません。")
        return

    print(f"\n'{file_path}' を読み込み中...")
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            raw_text = f.read()
        print(f"'{file_path}' の読み込みに成功しました。")
    except Exception as e:
        print(f"エラー: ファイル '{file_path}' の読み込みに失敗しました: {e}")
        return

    # --- 必要最小限の前処理 ---
    processed_text = re.sub(r"<br\s*/?>", "\n", raw_text, flags=re.IGNORECASE)
    processed_text = re.sub(r"\n\s*\n", "\n\n", processed_text)
    if "digipanp" in file_path.lower():
        print("digiPanpファイル向けの前処理を適用中...")
        processed_text = re.sub(r"<img\s+[^>]*>", "", processed_text, flags=re.IGNORECASE)
        processed_text = re.sub(r"\n\n0\n\n", "\n\n", processed_text)

    print(f"'{file_path}' の前処理完了。Geminiで整形中...")

    # --- Geminiへのプロンプト作成 ---
    system_instruction = "You are an expert in document structure analysis. Read the provided text and identify meaningful breaks where the topic or section changes."

    user_prompt = f"""Read the following text and insert a special marker symbol {marker} at the end of each meaningful section or unit. Examples of meaningful breaks include the end of a chapter, the end of a major section, the end of a faculty profile, or a significant topic change.

Constraints:
- DO NOT change the original text content except for inserting the marker symbol `{marker}`.
- Maintain the Markdown formatting (headers, lists, tables, etc.) as much as possible.
- The marker symbol should be inserted on a new line by itself (e.g., \\n{marker}\\n).
- Do not consider the original text length when deciding where to insert markers; base your decision on the meaning and structure of the text.

Text:
---
{processed_text}
---

Please provide the text with the {marker} marker inserted at the appropriate breaks."""

    # --- Gemini API呼び出し ---
    try:
        model = genai.GenerativeModel(model_name)  # モデル名を修正
        print(f"Geminiモデル '{model_name}' をロードしました。")

        print("Gemini API呼び出し中...")
        response = model.generate_content(
            [{"role": "user", "parts": [{"text": system_instruction + "\n\n" + user_prompt}]}],  # Gemini APIの形式に合わせる
        )

        if hasattr(response, "text"):
            generated_text = response.text
            print("Gemini API 呼び出し成功。")
        else:
            print(f"エラー: Geminiからの応答にテキストが含まれていません。")
            if hasattr(response, "prompt_feedback") and hasattr(response.prompt_feedback, "block_reason"):
                print(f"Prompt was blocked due to: {response.prompt_feedback.block_reason}")
            generated_text = None

    except Exception as e:
        print(f"エラー: Gemini API 呼び出しに失敗しました: {e}")
        generated_text = None

    if generated_text is None:
        print(f"'{file_path}' のGemini処理に失敗しました。")
        return

    # --- 生成結果の後処理 ---
    if generated_text.strip().startswith("```markdown"):
        generated_text = generated_text.strip()[len("```markdown") :].strip()
        if generated_text.strip().endswith("```"):
            generated_text = generated_text.strip()[: -len("```")].strip()

    # --- 整形後のファイルを保存 ---
    print(f"Geminiによる整形結果を '{output_path}' に保存中...")
    try:
        # 出力ディレクトリが存在するか確認し、存在しない場合は作成
        output_dir = os.path.dirname(output_path)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"出力ディレクトリ '{output_dir}' を作成しました。")

        with open(output_path, "w", encoding="utf-8") as f:
            f.write(generated_text)
        print(f"整形結果を '{output_path}' に保存しました。")
    except Exception as e:
        print(f"エラー: 整形結果の保存に失敗しました: {e}")
        return

    print(f"'{output_path}' を開いて、'{marker}' が適切に挿入されているか確認してください。")


# --- ファイルパスの設定 ---
# 現在の作業ディレクトリを基準にした相対パスを使用
base_dir = os.getcwd()
gakusoku_file_path = os.path.join(base_dir, "RAG", "kic-gakusoku.md")
digipanp_file_path = os.path.join(base_dir, "RAG", "KIC_digiPanp.md")

# LLMで整形したファイルを保存するパス
digipanp_marked_path = os.path.join(base_dir, "RAG", "KIC_digiPanp_marked_gemini.md")
gakusoku_marked_path = os.path.join(base_dir, "RAG", "kic-gakusoku_marked_gemini.md")


# --- 処理実行 ---

# APIキーが設定されている場合のみ実行
if GOOGLE_API_KEY and api_configured:
    print("\n--- Gemini API を使用したファイル整形処理を実行 ---")

    # digiPanpファイルの処理
    process_file_with_gemini(digipanp_file_path, digipanp_marked_path, MODEL_NAME, CHUNK_SEPARATOR_MARKER)

    # 学則ファイルの処理 
    process_file_with_gemini(gakusoku_file_path, gakusoku_marked_path, MODEL_NAME, CHUNK_SEPARATOR_MARKER)

    print("\n--- Geminiで整形したファイルを確認してください ---")
    print(f"- '{digipanp_marked_path}'")
    print(f"- '{gakusoku_marked_path}'")
    print("\n--- 処理完了 ---")
else:
    print("\nAPIキーが設定されていないか、API設定に失敗したため、Gemini API を使用したファイル処理はスキップされました。")


Gemini API configured successfully.

--- Gemini API を使用したファイル整形処理を実行 ---

'e:\Docker\colab-runtime\content\AI-E-03\RAG\KIC_digiPanp.md' を読み込み中...
'e:\Docker\colab-runtime\content\AI-E-03\RAG\KIC_digiPanp.md' の読み込みに成功しました。
digiPanpファイル向けの前処理を適用中...
'e:\Docker\colab-runtime\content\AI-E-03\RAG\KIC_digiPanp.md' の前処理完了。Geminiで整形中...
Geminiモデル 'gemini-2.5-flash-preview-04-17' をロードしました。
Gemini API呼び出し中...
Gemini API 呼び出し成功。
Geminiによる整形結果を 'e:\Docker\colab-runtime\content\AI-E-03\RAG\KIC_digiPanp_marked_gemini.md' に保存中...
整形結果を 'e:\Docker\colab-runtime\content\AI-E-03\RAG\KIC_digiPanp_marked_gemini.md' に保存しました。
'e:\Docker\colab-runtime\content\AI-E-03\RAG\KIC_digiPanp_marked_gemini.md' を開いて、'[CHUNK_SEPARATOR]' が適切に挿入されているか確認してください。

'e:\Docker\colab-runtime\content\AI-E-03\RAG\kic-gakusoku.md' を読み込み中...
'e:\Docker\colab-runtime\content\AI-E-03\RAG\kic-gakusoku.md' の読み込みに成功しました。
'e:\Docker\colab-runtime\content\AI-E-03\RAG\kic-gakusoku.md' の前処理完了。Geminiで整形中...
Geminiモデル 'gemini-2.5-flash-previ

### Qwen3による分割準備

#### モデル開放

In [ ]:
import torch
import gc

if 'model' in globals() and model is not None:
    print(f"アンロード対象モデル: {type(model)}")
    del model
    model = None
    print("モデル参照を削除しました。")

if 'tokenizer' in globals() and tokenizer is not None:
    del tokenizer
    tokenizer = None
    print("トークナイザー参照を削除しました。")

gc.collect()
print("ガベージコレクションを実行しました。")

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("CUDAキャッシュをクリアしました。")

print("\n--- 以前のモデルをアンロードしました ---")
# この後、nvidia-smiなどでVRAMが解放されたか確認すると良い

#### 4B

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

new_model_name = "Qwen/Qwen3-4B-AWQ" # 正しいAWQ版モデル名を確認してください
print(f"\n新しいモデル '{new_model_name}' をロード中...")

try:
    new_tokenizer = AutoTokenizer.from_pretrained(
        new_model_name,
        trust_remote_code=True
    )
    print(f"Tokenizer for '{new_model_name}' loaded successfully.")

    # YaRN設定 (4Bモデルのネイティブコンテキスト長が32768の場合)
    rope_scaling_config = {"type": "yarn", "factor": 4.0, "original_max_position_embeddings": 32768} # 必要に応じて調整

    new_model = AutoModelForCausalLM.from_pretrained(
        new_model_name,
        device_map="auto",
        trust_remote_code=True,
        rope_scaling=rope_scaling_config, # YaRN設定
        # attn_implementation="flash_attention_2", # 利用可能なら
    )
    print(f"Model '{new_model_name}' loaded successfully. Device: {new_model.device}")
    if hasattr(new_model.config, 'rope_scaling') and new_model.config.rope_scaling is not None:
        print(f"YaRN (rope_scaling) 設定: {new_model.config.rope_scaling}")
        if hasattr(new_model.config, 'max_position_embeddings'):
             print(f"モデルの最大コンテキスト長 (YaRN考慮): {new_model.config.max_position_embeddings}")
    else:
        print("警告: YaRN (rope_scaling) 設定が有効になっていない可能性があります。")


    # 新しいモデルとトークナイザーをグローバル変数に再代入 (元の変数名を使う場合)
    model = new_model
    tokenizer = new_tokenizer

except Exception as e:
    print(f"Error loading new model '{new_model_name}': {e}")
    model = None # ロード失敗時はNoneに
    tokenizer = None

In [ ]:
# --- 最初の1回だけ実行する部分 (モデルロード以外) ---
import torch
import re
import os

# ... (デバイス設定など、前回のコードから) ...
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


# --- LLMを使ったチャンクマーカー挿入関数 ---
CHUNK_SEPARATOR_MARKER = "[CHUNK_SEPARATOR]"

def process_file_with_qwen(file_path, output_path, model, tokenizer, marker=CHUNK_SEPARATOR_MARKER):
    if not os.path.exists(file_path):
        print(f"エラー: '{file_path}' が見つかりません。スキップします。")
        return
    if model is None or tokenizer is None:
         print("エラー: Qwen3モデルまたはトークナイザーがロードされていません。処理をスキップします。")
         return

    print(f"\n'{file_path}' を読み込み中...")
    with open(file_path, "r", encoding="utf-8") as f:
        raw_text = f.read()

    processed_text = re.sub(r'<br\s*/?>', '\n', raw_text, flags=re.IGNORECASE)
    processed_text = re.sub(r'\n\s*\n', '\n\n', processed_text)
    if 'digipanp' in file_path.lower():
        print("digiPanpファイル向けの前処理を適用中...")
        processed_text = re.sub(r'<img\s+[^>]*>', '', processed_text, flags=re.IGNORECASE)
        processed_text = re.sub(r'\n\n0\n\n', '\n\n', processed_text)

    print(f"'{file_path}' の前処理完了。Qwen3で整形中...")

    system_prompt = "あなたは文書構造を理解し、テキストの意味的な区切りを特定する専門家です。"
    user_prompt = f"""以下のテキストを読み、意味的なまとまりの区切りと思われる箇所（例：章の終わり、節の終わり、重要なトピックの変わり目など）に、特別なマーカー記号 {marker} を挿入してください。

制約条件：
- マーカー記号 `{marker}` 以外、**元のテキストの内容を一切変更しないでください**。
- Markdown形式（ヘッダー、リスト、テーブルなど）をできるだけ維持してください。
- マーカー記号は、新しい行に単独で挿入してください（例：\\n{marker}\\n）。
- 元のテキストの長さは考慮せず、意味的な区切りだけを判断してください。

テキスト：
---
{processed_text}
---

マーカー記号が挿入されたテキストを生成してください。""" # プロンプトの最後に指示を繰り返すことも検討

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # process_file_with_qwen 関数内の apply_chat_template 呼び出し部分
    prompt_text_for_qwen = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    # --- Qwen3によるテキスト生成 ---

    # モデルの実際の最大コンテキスト長を取得 (YaRN有効時を反映)
    try:
        # model.config.max_position_embeddings が YaRN 設定を反映するはず
        # トークナイザーの model_max_length は、YaRN有効化前の値のままのことがある
        max_tokens_model_can_handle = model.config.max_position_embeddings
        if max_tokens_model_can_handle is None: # 古い transformers などで取得できない場合
            print("警告: model.config.max_position_embeddings から最大コンテキスト長を取得できませんでした。tokenizer.model_max_length を使用します。")
            max_tokens_model_can_handle = tokenizer.model_max_length
    except AttributeError:
        print("警告: model.config.max_position_embeddings が存在しません。tokenizer.model_max_length を使用します。")
        max_tokens_model_can_handle = tokenizer.model_max_length

    print(f"モデルが処理可能な最大トークン長 (YaRN考慮): {max_tokens_model_can_handle}")


    input_token_ids = tokenizer(prompt_text_for_qwen, return_tensors="pt").input_ids
    input_token_count = input_token_ids.shape[1] # input_ids[0] ではなく shape[1] を使う (バッチサイズ1なので同じだが)

    max_gen_tokens = max_tokens_model_can_handle - input_token_count

    if max_gen_tokens <= 0:
         print(f"エラー: 入力テキストが長すぎ({input_token_count} トークン)、モデルが処理可能な最大コンテキスト長({max_tokens_model_can_handle})を超えています。")
         print("モデルロード時のYaRN設定が正しいか、または入力テキストを分割する必要があるか確認してください。")
         return

    # 生成パラメータ (do_sample=False の場合、temperatureなどは不要)
    generation_kwargs = {
        "max_new_tokens": max_gen_tokens,
        "eos_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.eos_token_id, # tokenizer.eos_token_id が None の場合、エラーになるので注意
        "do_sample": False,
    }
    # tokenizer.eos_token_id が None の場合のフォールバック
    if generation_kwargs["eos_token_id"] is None and hasattr(tokenizer, 'eod_id'):
        generation_kwargs["eos_token_id"] = tokenizer.eod_id
        generation_kwargs["pad_token_id"] = tokenizer.eod_id
    if generation_kwargs["eos_token_id"] is None:
        print("警告: eos_token_id が設定できませんでした。生成が意図通りに終了しない可能性があります。")
        # 特定のID (例: <|endoftext|>) を使うか、エラーにするかなど、モデルに合わせて対応が必要


    model_inputs = {"input_ids": input_token_ids.to(model.device)} # 既にテンソルなので再度tokenizerに渡さない

    try:
        print(f"Qwen3で生成中 (入力トークン数: {input_token_count}, モデル処理可能最大長: {max_tokens_model_can_handle}, 生成最大トークン数: {max_gen_tokens})...")
        with torch.no_grad(): # 推論時は勾配計算不要
            outputs = model.generate(
                **model_inputs,
                **generation_kwargs
            )

        response_ids = outputs[0][model_inputs["input_ids"].shape[-1]:]
        generated_text = tokenizer.decode(response_ids, skip_special_tokens=False)

        im_end_token_id = tokenizer.convert_tokens_to_ids("<|im_end|>") # Qwenの特殊トークンIDを取得
        if im_end_token_id is not None: # Noneチェックを追加
            im_end_token = tokenizer.decode(im_end_token_id)
            if im_end_token and im_end_token in generated_text: # im_end_token が空文字列でないことも確認
                 generated_text = generated_text.split(im_end_token)[0]

        print(f"Qwen3による整形結果を '{output_path}' に保存中...")
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(generated_text)
        print("保存完了。")
        print(f"'{output_path}' を開いて、'{marker}' が適切に挿入されているか確認してください。")

    except Exception as e:
        print(f"\nQwen3生成中にエラーが発生しました: {e}")
        print("VRAM不足やその他の原因が考えられます。モデルロード設定やGPUリソースを確認してください。")


# --- ファイルパスの設定 ---
gakusoku_file_path = "AI-E-03/RAG/kic-gakusoku.md"
digipanp_file_path = "AI-E-03/RAG/KIC_digiPanp.md"
digipanp_marked_path = "AI-E-03/RAG/KIC_digiPanp_marked.md"
gakusoku_marked_path = "AI-E-03/RAG/kic-gakusoku_marked.md"


# --- 処理実行 ---
# model および tokenizer 変数が他のセルで定義され、YaRNが有効化されていることを想定
if 'model' in globals() and 'tokenizer' in globals() and model is not None and tokenizer is not None:
    print("\nモデルとトークナイザーがロード済みであることを確認しました。ファイル処理を実行します。")
    # モデルロード時にYaRNが有効になっているか確認
    if hasattr(model.config, 'rope_scaling') and model.config.rope_scaling is not None:
        print(f"YaRN (rope_scaling) 設定がモデルコンフィグに検出されました: {model.config.rope_scaling}")
    else:
        print("警告: モデルコンフィグにYaRN (rope_scaling) 設定が見つかりません。長いコンテキストを扱えない可能性があります。")

    process_file_with_qwen(digipanp_file_path, digipanp_marked_path, model, tokenizer, CHUNK_SEPARATOR_MARKER)
    # process_file_with_qwen(gakusoku_file_path, gakusoku_marked_path, model, tokenizer, CHUNK_SEPARATOR_MARKER) # 必要なら

    print("\n--- LLMで整形したファイルを確認してください ---")
    # ... (確認メッセージ)
else:
    print("\nモデルのロードに失敗したか、モデル/トークナイザーの変数が見つかりませんでした。ファイル処理はスキップされました。")

### チャンク化

In [16]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

In [17]:
import os
import re
# Text Splitter を使用するためにインポート
from langchain_text_splitters import RecursiveCharacterTextSplitter

# チャンク化マーカー記号の定義 (Geminiに挿入させたものと同じ)
CHUNK_SEPARATOR_MARKER = "[CHUNK_SEPARATOR]"

# --- 処理対象ファイルパスの設定 ---

digipanp_marked_path = "AI-E-03/RAG/KIC_digiPanp_marked_gemini.md"
# 以前章ごとに分割・クリーニングした学則ファイル
gakusoku_cleaned_path = "AI-E-03/RAG/kic-gakusoku_marked_gemini.md"

# 最終的なチャンク化されたドキュメントリストを格納する変数
final_chunked_documents = []

# --- 1. Geminiで整形したdigiPanpファイルを読み込み、チャンク化 ---

if os.path.exists(digipanp_marked_path):
    print(f"'{digipanp_marked_path}' を読み込み中...")
    try:
        with open(digipanp_marked_path, "r", encoding="utf-8") as f:
            digipanp_marked_text = f.read()
        print(f"'{digipanp_marked_path}' の読み込みに成功しました。")

        # チャンク化のパラメータ設定
        # Geminiが挿入したマーカーを最優先に使うが、fallbackも定義
        # チャンクサイズやオーバーラップは、RAGの検索精度やVRAMに合わせて調整
        chunk_size = 1024    # チャンクの最大長 (文字数) - VRAMに合わせて小さく
        chunk_overlap = 100 # オーバーラップの長さ - 文脈維持のため

        print(f"\n'{digipanp_marked_path}' をチャンク化中 (チャンクサイズ={chunk_size}, オーバーラップ={chunk_overlap}, 区切り='{CHUNK_SEPARATOR_MARKER}'などを優先)...")

        # Text Splitterのインスタンス作成
        # Geminiが挿入したマーカーを最優先の区切りとして指定
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len, # 文字数で長さを測る
            is_separator_regex=False, # マーカー記号は固定文字列として扱う
            separators=[
                CHUNK_SEPARATOR_MARKER, # これが最優先の区切りになる
                # その他のフォールバック区切り文字 (もしマーカー挿入が完璧でない場合や、マーカーのない部分のため)
                # Markdownのヘッダー、連続改行、改行、空白などを優先度の高い順に
                r"\n#+\s+.*?\n", # Markdown ヘッダー (##, ### など) - 正規表現なので is_separator_regex=True と組み合わせるか、個別に定義
                r"\n\n+",      # 連続する改行 (段落区切り)
                r"\n",         # 単一の改行 (行区切り)
                " ",          # 空白
                "",           # 最後の手段
            ]
        )

        # is_separator_regex=True を使う場合は、すべての separators を正規表現として書く必要があります。
        # マーカーを固定文字列、それ以外を正規表現として混ぜて使いたい場合は、少し工夫が必要です。
        # ここでは、シンプルにマーカーを最優先の固定文字列として、他の区切りはフォールバックとして定義します。
        # もしMarkdownヘッダーなどを正規表現で正確に区切りたい場合は、separatorsリスト全体を正規表現にするか、
        # マーカーの有無で splitter を切り替えるなどのロジックが必要になります。
        # 例として、マーカーを最優先し、後は一般的な改行・空白で分けるシンプル版にします。
        text_splitter_simple_separators = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            is_separator_regex=False, # マーカーは固定文字列
            separators=[
                CHUNK_SEPARATOR_MARKER, # マーカーを最優先
                "\n\n",                 # 段落区切り
                "\n",                   # 改行
                " ",
                "",
            ]
        )


        # チャンク化を実行
        # create_documents はリストを受け取るので、読み込んだテキストをリストに入れる
        gemini_marked_chunks = text_splitter_simple_separators.create_documents([digipanp_marked_text])

        # Textオブジェクトから文字列を抽出してリストに追加
        gemini_marked_chunk_contents = [chunk.page_content for chunk in gemini_marked_chunks]
        final_chunked_documents.extend(gemini_marked_chunk_contents)

        print(f"'{digipanp_marked_path}' から {len(gemini_marked_chunk_contents)} 件のチャンクを生成しました。")

    except Exception as e:
        print(f"エラー: '{digipanp_marked_path}' の処理中にエラーが発生しました: {e}")

else:
    print(f"警告: '{digipanp_marked_path}' が見つかりませんでした。このファイルはチャンク化されません。")


# --- 2. クリーニング済みの学則ファイルを読み込み、チャンク化 ---

if os.path.exists(gakusoku_cleaned_path):
    print(f"\n'{gakusoku_cleaned_path}' を読み込み中...")
    try:
        with open(gakusoku_cleaned_path, "r", encoding="utf-8") as f:
            gakusoku_cleaned_text = f.read()
        print(f"'{gakusoku_cleaned_path}' の読み込みに成功しました。")

        # 学則ファイルにはLLMマーカーがないので、章や条文を優先する区切り文字を使う
        # チャンクサイズやオーバーラップはdigiPanpと合わせるか、文書構造に合わせて調整
        # ここでは、学則の構造（章、条文）を考慮したseparatorsを使う
        chunk_size_gakusoku = 512 # 学則向けに調整しても良い
        chunk_overlap_gakusoku = 100

        print(f"\n'{gakusoku_cleaned_path}' をチャンク化中 (チャンクサイズ={chunk_size_gakusoku}, オーバーラップ={chunk_overlap_gakusoku}, 区切り='第X章', '第X条'などを優先)...")

        # Text Splitterのインスタンス作成 (学則向け)
        text_splitter_gakusoku = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size_gakusoku,
            chunk_overlap=chunk_overlap_gakusoku,
            length_function=len, # 文字数で長さを測る
            is_separator_regex=True, # 正規表現を使うためTrueに
            separators=[
                CHUNK_SEPARATOR_MARKER, # マーカーを最優先
                r"\n#?\s*第\d+章\s*.*?\n",
                 # 条文の開始
                r"\n第\d+条\s*.*?\n",
                 # 連続する改行 (段落区切り)
                r"\n\n+",
                # 単一の改行 (行区切り)
                r"\n",
                # その他空白など
                " ",
                "",
            ]
        )

        # チャンク化を実行
        gakusoku_chunks = text_splitter_gakusoku.create_documents([gakusoku_cleaned_text])

        # Textオブジェクトから文字列を抽出してリストに追加
        gakusoku_chunk_contents = [chunk.page_content for chunk in gakusoku_chunks]
        final_chunked_documents.extend(gakusoku_chunk_contents) # 最終リストに追加

        print(f"'{gakusoku_cleaned_path}' から {len(gakusoku_chunk_contents)} 件のチャンクを生成しました。")

    except Exception as e:
        print(f"エラー: '{gakusoku_cleaned_path}' の処理中にエラーが発生しました: {e}")

else:
    print(f"警告: '{gakusoku_cleaned_path}' が見つかりませんでした。このファイルはチャンク化されません。")


# --- 3. 最終的な統合チャンクリストの確認 ---

print(f"\n--- 最終的な統合チャンクリスト ---")
print(f"合計チャンク数: {len(final_chunked_documents)} 件")

# 最初のいくつかを表示して確認
print("\n--- 統合チャンクリストの冒頭の例 ---")
for i, chunk in enumerate(final_chunked_documents[:10]): # 最初の10件を表示
    print(f"\n--- チャンク {i+1} ---")
    print(f"内容 ({len(chunk)} 文字):\n")
    # 長すぎる場合は最初の数行だけ表示
    print('\n'.join(chunk.splitlines()[:15]))
    # あるいは、チャンク全体を表示 (短ければ)
    # print(chunk[:500] + '...' if len(chunk) > 500 else chunk)


# --- 4. この final_chunked_documents を使って埋め込み計算とベクトルデータベース化に進む ---
# SentenceTransformer モデルのロード (emb_model) や、ベクトルデータベースライブラリ (例: Chroma, Faiss) の準備
# 次のステップのコードはここ以降に記述します。

'AI-E-03/RAG/KIC_digiPanp_marked_gemini.md' を読み込み中...
'AI-E-03/RAG/KIC_digiPanp_marked_gemini.md' の読み込みに成功しました。

'AI-E-03/RAG/KIC_digiPanp_marked_gemini.md' をチャンク化中 (チャンクサイズ=1024, オーバーラップ=100, 区切り='[CHUNK_SEPARATOR]'などを優先)...
'AI-E-03/RAG/KIC_digiPanp_marked_gemini.md' から 93 件のチャンクを生成しました。

'AI-E-03/RAG/kic-gakusoku_marked_gemini.md' を読み込み中...
'AI-E-03/RAG/kic-gakusoku_marked_gemini.md' の読み込みに成功しました。

'AI-E-03/RAG/kic-gakusoku_marked_gemini.md' をチャンク化中 (チャンクサイズ=512, オーバーラップ=100, 区切り='第X章', '第X条'などを優先)...
'AI-E-03/RAG/kic-gakusoku_marked_gemini.md' から 19 件のチャンクを生成しました。

--- 最終的な統合チャンクリスト ---
合計チャンク数: 112 件

--- 統合チャンクリストの冒頭の例 ---

--- チャンク 1 ---
内容 (606 文字):

---
0

# 神戸情報大学院大学
情報技術研究科 情報システム専攻





社会の加速度的な変化に伴う事象は、

経験や基礎知識だけでは対応できない時代となっています。
近年、文部科学省では「総合的な探究の時間」を実施し、社会で求められる
力の育成を求めています。

--- チャンク 2 ---
内容 (427 文字):

[CHUNK_SEPARATOR]

# Contents

|トップメッセージ|03|
|-|-|
|教育コンセプト|05|
|育成人材像と活躍のフィールド|07|
|コースマップ|08|
|履修科目一覧|09|
|フルオンライン履修制度|11|
|主な教員一覧|13|
|コース紹介|22|

# 研究室紹介

--- チャンク 

In [18]:
#エンベディング
print("\n統合ドキュメントの埋め込みを計算中...")

batch_size_to_try = 12
document_embeddings = emb_model.encode(final_chunked_documents, batch_size=batch_size_to_try, show_progress_bar=True)
print("埋め込み計算完了。")


統合ドキュメントの埋め込みを計算中...


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

埋め込み計算完了。


#### 質問１

In [19]:
# Retrievalの実行
question =  "神戸情報大学院大学の学長の名前と主なプロフィール（経歴や役職）を3点挙げてください。"

query_embeddings = emb_model.encode([question], prompt_name="query")

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[76.73336029052734, 70.09751892089844, 75.80654907226562, 79.1156997680664, 75.51924896240234, 71.23345184326172, 71.1617431640625, 66.82563781738281, 67.62478637695312, 69.06987762451172, 64.29704284667969, 66.03050231933594, 63.55447006225586, 64.60484313964844, 66.13522338867188, 63.17173767089844, 63.02774429321289, 71.57255554199219, 67.18860626220703, 68.3012466430664, 69.18016815185547, 70.20384216308594, 71.43720245361328, 68.95332336425781, 69.95565032958984, 71.91834259033203, 64.84209442138672, 64.685302734375, 68.61988830566406, 61.917137145996094, 63.11894607543945, 68.99974060058594, 67.6261215209961, 73.64942169189453, 68.15306854248047, 69.98910522460938, 65.75389862060547, 66.71220397949219, 74.16929626464844, 70.51712036132812, 66.89803314208984, 68.76871490478516, 62.96842575073242, 63.844974517822266, 67.12525177001953, 66.9295425415039, 69.95671081542969, 70.75691223144531, 66.91551208496094, 67.09808349609375, 67.66845703125, 61.696468353271484, 75.34756469726562

In [20]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(final_chunked_documents[index], "\n\n")

取得したドキュメント1: (Score: 79.1156997680664)
[CHUNK_SEPARATOR]

# 探究型人材へとグレードアップすれば、
あなたの活躍フィールドは無限に

神戸情報大学院大学
学長代理
福岡 賢二

# PROFILE

博士\(学術\)専門分野:Computer & Communication \(コンピュータと情報コミュニケーション\)
スウィフト·エックスアイ株式会社 代表取締役社長
Tankyu X 代表取締役社長
KTGM合同会社 代表
IEEE\(米国電気電子学会\)Education Society会員

神戸情報大学院大学の母体である神戸電子専門学校は昭和33年に
設立されました。半世紀以上をかけて約2万人を産業界に輩出して
きたICT人材育成のパイオニアです。1995年阪神・淡路大震災の際
には、同校研究所が被災地の情報をパソコン通信やインターネットを
活用して整理·配信する情報拠点となり、当時大変話題となりました。
この時生まれた本学の「Social Innovation by ICT & Yourself」
\(ICTと人間力による社会課題解決\)というスピリッツは、今でも多く
の若者に影響を与えています。

近年、アフリカ等開発途上国が共通に抱える「貧富の差」解消のため
に、富が一部に集中しがちな資源ビジネスに頼らない産業開発、つま
り「Socio Economy」への転換が期待されています。そして比較的

参入障壁の少ないICTイノベーションに注目が集まっています。本学
ではこのようなトレンドをいち早く取り込み、独立行政法人国際協力機
構\(JICA\)等の協力を受けて平成25年度に「ICTイノベータコース」
を新設しました。

本学は社会の課題を発見し、IT技能や人間力を磨きつつ解決を目指
す「探究型人材」の育成を使命としています。なぜなら、このような人
材が社会で強く求められているからです。それは日本のみならず世
界的でも高まる一方です。是非みなさんも神戸情報大学院大学へ入
学され、自分と世界をより良い方向へ導くための一歩を、同じ志を持
つ仲間と共に踏み出されんことを、強く望んでおります。 


取得したドキュメント2: (Score: 76.73336029052734)
---
0

# 神戸情報

In [21]:
references = "\n".join(["* " + final_chunked_documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 神戸情報大学院大学の現学長の名前と主なプロフィール（経歴や役職）を3点挙げてください。"
response_off = generate_output_qwen3(query, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF with RAG: {response_off}")

Thinking OFF with RAG: 神戸情報大学院大学の現学長の名前は**炭谷 俊樹**様です。主なプロフィール（経歴や役職）は以下の3点です：

1. **東京大学理学系研究科修士課程修了**（物理学専攻）  
2. **ビジネス・ブレークスルー大学大学院客員教授**（経営管理専攻・問題解決思考）  
3. **ラーンネット・グローバルスクール代表**


In [34]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

0.0


#### 質問2

In [22]:
# Retrievalの実行
question =  "神戸情報大学院大学に入学する場合、学則第30条および別表1に定められている入学金と年間の授業料はそれぞれいくらですか？"

query_embeddings = emb_model.encode([question], prompt_name="query")

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[73.96046447753906, 67.84422302246094, 68.99053192138672, 72.49700164794922, 70.62515258789062, 67.873291015625, 69.26591491699219, 64.85391235351562, 64.87320709228516, 63.96805191040039, 67.4737319946289, 68.29173278808594, 64.03221130371094, 64.19709014892578, 66.41323852539062, 63.13868713378906, 62.57064437866211, 71.29381561279297, 71.0172119140625, 63.92611312866211, 63.492733001708984, 65.53244018554688, 65.13324737548828, 63.12240219116211, 64.00186920166016, 65.39584350585938, 60.83317947387695, 64.71478271484375, 65.48827362060547, 60.224002838134766, 62.160037994384766, 65.12980651855469, 63.36377716064453, 69.17093658447266, 63.661705017089844, 65.65830993652344, 60.83595657348633, 65.5403060913086, 69.04161834716797, 64.13674926757812, 63.87223434448242, 64.4083023071289, 62.694828033447266, 60.42071533203125, 62.410186767578125, 60.46118927001953, 63.129032135009766, 65.15848541259766, 63.887481689453125, 61.727142333984375, 65.12053680419922, 59.76812744140625, 70.6615

In [23]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(final_chunked_documents[index], "\n\n")

取得したドキュメント1: (Score: 77.84923553466797)
CHUNK_SEPARATOR]

# 別表1 入学金、授業料、入学検定料、その他の費用

|区 分|金 額|
|-|-|
|入学金|200,000円|
|授業料\(1年当たり\)|1,400,000円|
|施設·設備費\(1年当たり\)|200,000円|
|入学検定料|30,000円|

[CHUNK_SEPARATOR]

# 別表2 在籍料

|区 分|金 額|
|-|-|
|在籍料\(1期当たり\)|20,000円|

[CHUNK_SEPARATOR]

# 別表3 科目等履修生の授業料、その他の費用

|区 分|金 額|
|-|-|
|登録料|10,000円|
|授業料\(1単位当たり\)|40,000円|
|実験科目加算額\(1単位当たり\)|40,000円|

[CHUNK_SEPARATOR]

# 別表 4 研究生の研究指導料、選考料

|区 分|金 額|
|-|-|
|研究指導料\(1期当たり\)|80,000円|

6

|選考料|20,000円|
|-|-|

※ 但し、研究指導の内容に応じて、研究指導料を減額する場合がある。

7

[CHUNK_SEPARATOR] 


取得したドキュメント2: (Score: 77.30162048339844)
CHUNK_SEPARATOR]

第8章 入学検定料、入学金及び授業料等

# \(入学検定料、入学金及び授業料等\)

第30条 入学検定料、入学金及び授業料等は、別表1の通りとする。

2
入学検定料、入学金及び授業料等の免除あるいは減免、及び特段の理由のある者へ
の奨学の方法については別途定めることができる。

第31条 既に納めた前条の学納金は、事情の如何に拘らず、これを返還しない。但し、別途
指定の期日までに入学辞退の意思表示をした者\(推薦入学試験\(これに類する入学試験
を含む。\)に合格して本大学院と在学契約を締結した者を除く。\)については、原則と
して、既に納めた入学金以外の学納金の返還に応じる。

第32条 休学期間中は、在籍料を納付しなければならない。

2
在籍料は、別表2の通りとする。

[CHUNK_SEPARATO 


取得したドキュメント3: (Score: 77.195

In [24]:
references = "\n".join(["* " + final_chunked_documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 神戸情報大学院大学に入学する場合、学則第30条および別表1に定められている入学金と年間の授業料はそれぞれいくらですか？"
response_off = generate_output_qwen3(query, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF with RAG: {response_off}")

Thinking OFF with RAG: 神戸情報大学院大学に入学する場合、学則第30条および別表1に定められている費用は以下の通りです。

- **入学金**: 200,000円  
- **年間の授業料**: 1,400,000円  

これらは、別表1に記載されている内容に基づいています。


#### 質問3

In [25]:
# Retrievalの実行
question =  "神戸情報大学院大学において「フルオンライン履修制度」を利用して正規課程を修了した場合、学則に基づき授与される学位の正式名称は何ですか？"

query_embeddings = emb_model.encode([question], prompt_name="query")

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[72.7366714477539, 72.38206481933594, 69.44607543945312, 74.16433715820312, 71.88084411621094, 68.48347473144531, 70.36610412597656, 67.7896957397461, 68.58222198486328, 68.81306457519531, 66.27952575683594, 68.90411376953125, 66.33866119384766, 66.44982147216797, 68.60908508300781, 64.09242248535156, 65.33769226074219, 81.40701293945312, 72.41220092773438, 63.394771575927734, 65.83049011230469, 67.80870819091797, 66.90342712402344, 66.13384246826172, 67.02267456054688, 68.17760467529297, 66.0386962890625, 66.20551300048828, 66.86253356933594, 62.47135543823242, 64.30550384521484, 66.95384979248047, 64.48304748535156, 70.84044647216797, 65.75733184814453, 68.08390808105469, 63.84351348876953, 67.67739868164062, 72.524169921875, 67.12199401855469, 62.261356353759766, 66.96660614013672, 62.12529373168945, 63.37192153930664, 65.8144302368164, 62.18590545654297, 66.61044311523438, 66.47903442382812, 64.30729675292969, 66.37816619873047, 67.11925506591797, 61.79116439819336, 71.26445770263

In [26]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(final_chunked_documents[index], "\n\n")

取得したドキュメント1: (Score: 81.40701293945312)
[CHUNK_SEPARATOR]

フルオンライン履修制度

# 正規課程



# 「働きながら」「完全オンライン」で修士号を取得\!

この度神戸情報大学院大学では、正規課程\(専門職修士課程\)をオンラインにより受講することによって、修了に必
要なすべての科目を取得し、修士の学位\[情報システム修士\(専門職\)」を取得するフルオンライン履修制度を開始
します。これにより、今までお住まいが遠方であったり、仕事が忙しく通学が困難だった方も、本学の修士号が取得
できるようになりました。

# 社会課題解決型
デジタル人材を目指す

本学では、多くの学生が自身の強みやこれまでの経験とICTを掛け合わせ、社会課題解決に挑戦
しています。卒業後は、IT業界のみならず、国際開発や様々なビジネスの分野で活躍しています。

# 世界中から集まった仲間と
オンラインで学ぶ

開学以降、中国やベトナム·ネパールなどアジアの学生のみならず、ルワンダ·タンザニア·モザ
ンビークなどアフリカ諸国や、アフガニスタン·シリアなど中東地域から多くの学生を受け入れ
ており、これまで受け入れた学生の出身国は100か国を超えます。自宅にいながら世界中から集
まった学生とオンラインで共に学ぶことができます。

# KIC 独自のメソッド
探究 ×ICT で学ぶ

炭谷俊樹学長が提唱し、本学に導入した「探究」精神は、本学での研究活動をはじめとした全ての
学びに根差すものです。人から与えられた課題ではなく、情熱をもって取り組める課題を自分で
見つけ、自分の得意技を磨き、活かしながら解決できる。そのような「探究」人材に必要な能力を
養い、手法をマスターしていただきます。

# 学生の声









# 利便性も内容も充実
仕事との両立に最適な環境です

仕事をしながら学生をする私のような人にとって、オンライ
ンという環境はたいへん相性がいいことを実感しました。
日々の授業や発表だけでなく、修士論文のための調査やイン
タビューにおいてもオンライン環境が効果的でした。KIC は
先生方や事務の方々のサポートがすばらしいので、充実した
2年間を得られることと思います。

11 Kobe Institute of Comput

In [27]:
references = "\n".join(["* " + final_chunked_documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 神戸情報大学院大学において「フルオンライン履修制度」を利用して正規課程を修了した場合、学則に基づき授与される学位の正式名称は何ですか？"
response_off = generate_output_qwen3(query, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF with RAG: {response_off}")

Thinking OFF with RAG: 神戸情報大学院大学において「フルオンライン履修制度」を利用して正規課程を修了した場合、学則に基づき授与される学位の正式名称は「情報システム修士（専門職）」です。


#### 質問4

In [28]:
# Retrievalの実行
question =  "神戸情報大学院大学では「100カ国を超える国々からの留学生」を受け入れていることが特徴の一つです。この大学の特徴は、学則に定められる入学資格のどの項目によって具体的に支えられていると考えられますか？関連する項目を複数挙げ、その理由を説明してください。"

query_embeddings = emb_model.encode([question], prompt_name="query")

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[77.13321685791016, 69.0195083618164, 72.17252349853516, 75.25276184082031, 77.3584213256836, 70.21322631835938, 71.35956573486328, 65.19817352294922, 67.43772888183594, 68.61251831054688, 67.54830932617188, 68.634033203125, 65.57353973388672, 66.40359497070312, 68.1423110961914, 64.30738830566406, 64.38411712646484, 73.90599060058594, 71.62615203857422, 62.87370300292969, 63.36369323730469, 67.63590240478516, 66.3619155883789, 65.96111297607422, 65.75086975097656, 70.41771697998047, 64.30110168457031, 67.18574523925781, 66.41351318359375, 61.30301284790039, 62.501548767089844, 68.04669952392578, 63.4603271484375, 71.43395233154297, 63.94472122192383, 66.43206024169922, 62.519775390625, 67.54508972167969, 72.49945831298828, 64.4850845336914, 63.019805908203125, 65.66040802001953, 63.846458435058594, 61.49400329589844, 64.20842742919922, 62.63080978393555, 67.173828125, 67.5746078491211, 68.3590316772461, 63.888465881347656, 66.72490692138672, 60.38140106201172, 72.8509292602539, 65.33

In [29]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(final_chunked_documents[index], "\n\n")

取得したドキュメント1: (Score: 77.855224609375)
---
# 第1章 総 則

# \(目 的\)

# 神戸情報大学院大学学則

第1条
神戸情報大学院大学\(以下「本大学院」という。\)は、人間力を有する高度 ICT
人材の育成を目的とする。

\(自己評価\)

第2条
2
本大学院は、前条の目的を達成するため、教育研究活動の点検及び評価を行う。
前項の点検及び評価の組織及び方法は、これを別に定める。

\(課 程\)

第3条
本大学院に設置する課程は、専門職学位課程とする。

[CHUNK_SEPARATO 


取得したドキュメント2: (Score: 77.3584213256836)
[CHUNK_SEPARATOR]

# 世界中で学びを求めている方々に対して、
世界でもユニークな学びの環境を

本学は2005年の開学以来、関係皆様からの温かい御理解·御支援と教職員
一同による不断の努力を通じて、「人間力を有する高度情報通信技術\(ICT\)
人材の育成」を一貫して行ってまいりました。複雑多様化する国際社会の問
題にも常に目を配り、海外からの留学生受入にも特段の力を注いでまいりま
した。その結果として、国内外でもユニークな専門職大学院として認知度が
徐々に高まり、多方面からの御支持を頂戴する機会が増えてまいりました。

本学は、学生皆様に対して、ICTを操る技術力を伝授するのみならず、何のた
めにICTを利活用し、その結果として社会課題の解決にどのように貢献でき
るのか、という現代社会で最も求められている根本的な能力を身に着けてい
ただくことを、目標としています。そのためには、常日頃から最新技術に関す
る関心を持って自己研鑽を行い、国内外の社会課題に対する「なぜ?」という
探究心を磨くことが大変重要です。そして何より、これらを「ジブンゴト」とし
て捉えて、課題解決に果敢にチャレンジしていく積極性が必要です。

言うは易く行うは難し。そんな複合的で難しいことを、簡単に出来るわけがあ
りません。私自身も民間企業、独立行政法人、国際開発金融機関などでの実
務者としての勤務経験を踏まえつつ、いまだに試行錯誤しながら様々な仮説
検証を行い続ける毎日です。しかしながら、本学の大きな特徴のひとつは、こ
のように現代社会で最も求められてい

In [30]:
references = "\n".join(["* " + final_chunked_documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 神戸情報大学院大学では「100カ国を超える国々からの留学生」を受け入れていることが特徴の一つです。この大学の特徴は、学則に定められる入学資格のどの項目によって具体的に支えられていると考えられますか？関連する項目を複数挙げ、その理由を説明してください。"
response_off = generate_output_qwen3(query, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF with RAG: {response_off}")

Thinking OFF with RAG: 神戸情報大学院大学が「100カ国を超える国々からの留学生」を受け入れているという特徴は、学則に定められた**入学資格の項目**によって具体的に支えられています。以下に関連する項目を挙げて、その理由を説明します。

---

### 1. **第18条（入学資格）の（2）外国において学校教育における16年の課程を修了した者**

#### 理由：
この項目は、**外国の教育制度において16年間の学校教育を修了した者**が入学資格を有することを明記しています。これは、**海外からの留学生**が日本の大学院に入学するための重要な基準であり、国際的な教育背景を持つ学生を歓迎する姿勢を示しています。  
また、日本の大学院では、通常は日本の大学を卒業した者（第18条（1））が対象ですが、国際的な教育を受けた者が入学できるようにすることで、**多様な文化や背景を持つ学生の受け入れが可能**になります。これにより、100カ国を超える国々からの留学生を受け入れるという特徴が支えられています。

---

### 2. **第18条（入学資格）の（5）その他、大学を卒業したと同等以上の学力があると本大学院において認められた者**

#### 理由：
この項目は、**日本の大学を卒業したと同等以上の学力があると認められた者**が入学資格を有することを規定しています。これは、**海外の大学を卒業した者や、日本の大学を卒業していないが同等以上の学力を持つ者**にも入学の機会を提供しています。  
特に、海外の大学を卒業した者や、国際的な教育制度において学力を証明した者（例：海外の大学の学位、または国際的な資格）は、この項目を通じて入学資格を得ることができます。これにより、**国際的な教育背景を持つ学生の受け入れが可能**となり、100カ国を超える国々からの留学生を受け入れるという特徴が支えられています。

---

### 結論：
神戸情報大学院大学が「100カ国を超える国々からの留学生」を受け入れるという特徴は、**第18条（入学資格）の（2）と（5）**の項目によって具体的に支えられています。  
- （2）は、**海外の教育制度において16年間の学校教育を修了した者**を対象としており、国際的な教育背景を持つ学生の受け入れを可能にしていま

#### 質問5

In [31]:
# Retrievalの実行
question =  "神戸情報大学院大学に在籍する高原敏竜特任教授は、国際機関のプロジェクト調整員や企画調査員として複数の国での活動経験があります。これらの国々の一般的な開発課題をいくつか挙げてください。その上で、高原特任教授が専門とする分野が、これらの開発課題の解決にどのように貢献できる可能性があるか、具体例を交えて説明してください。"

query_embeddings = emb_model.encode([question], prompt_name="query")

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[75.6363754272461, 71.81573486328125, 75.06741333007812, 75.24454498291016, 76.62532806396484, 73.84797668457031, 72.2383804321289, 71.47856903076172, 72.17637634277344, 72.15621185302734, 65.60359191894531, 70.9036636352539, 69.38259887695312, 71.22017669677734, 71.74536895751953, 70.07562255859375, 70.35017395019531, 71.24279022216797, 68.6019515991211, 67.48643493652344, 77.67121124267578, 75.02464294433594, 73.9659194946289, 73.94100189208984, 74.01785278320312, 75.06344604492188, 71.58330535888672, 69.88758850097656, 71.12117004394531, 65.725830078125, 65.92926025390625, 70.3008804321289, 65.66312408447266, 75.07464599609375, 72.66679382324219, 74.0111312866211, 68.30397033691406, 69.70807647705078, 76.61710357666016, 73.93387603759766, 66.11080932617188, 72.32518768310547, 66.23703002929688, 69.5081787109375, 72.159423828125, 74.29803466796875, 75.8647232055664, 72.67213439941406, 69.83580780029297, 71.11346435546875, 71.89398193359375, 63.53141784667969, 73.81856536865234, 76.8

In [32]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(final_chunked_documents[index], "\n\n")

取得したドキュメント1: (Score: 77.67121124267578)
[CHUNK_SEPARATOR]

高原 敏竜
特任教授



TAKAHARA, Toshiro

# 主要なテーマ·業務などの専門分野

ICTを活用したビジネスの価値づくり
MOT\(技術経営\)
技術者、技術マネジャー、DXリーダー人材育成

主な研究 実務実績

自動設計CADシステム\(エキスパートシステム\)の研究開発

大型機、ミニコン、WS、PCでのソフト開発、ソフトウェア工学

技術研修の企画運営\(UNIX、C、ネットワーク、問題解決手法など\)

リーダー研修の企画運営\(MOT=技術経営、技術リーダー研修、SEリーダー研修、ITリーダー研修など\)

戦略的研修の企画運営\(幹部候補研修、キャリア研修、スキルチェンジ研修、新規事業創造をテーマとする研修\)

社内人材育成体系の構築、インストラクショナルデザインなど

「図解でわかるソフトウェア開発のすべて」\(共著\)日本実業出版 2000年\(7万部\)

「ITビジネスコンサルタント」\(共著\)翔泳社 2001年\(7千部\)

# プロフィール:

博士\(知識科学=経営学系\)

技術士\(情報工学\)

中小企業診断士

キャリアコンサルタント

京都大学工学部数理工学科卒業、同大学院修士課程修了。

パナソニック入社、本社研究部門で情報通信技術の研究に従事、人材開発部門にてさまざまな社内研修の企画運営を担当。パナソ
ニック在籍中に北陸先端科学技術大学院大学知識科学研究科の博士後期課程修了。2018年より神戸情報大学院大学教授。

情報処理学会、組織学会、日本技術士会、大阪府中小企業診断協会、日本キャリアコンサルタント協会会員

学生へのメッセージ

ICT技術が急激に進化する中で、新しいICT技術の開発や習得は重要ですが、それらをどのように活用して社会の役に立てていく
のかもまた重要な課題です。技術力を効果的活用して、ビジネスの構築や社会課題の解決に貢献するなど、顧客や社会のための
価値づくりをしていくことが求められています。

これらをしっかり考えて、新しいシステムやビジネスを開発できる人材になってほしいと考えています。そのために、技術を習得す
ることはもちろんですが、価値づくりの考え方につい

In [33]:
references = "\n".join(["* " + final_chunked_documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 神戸情報大学院大学に在籍する高原敏竜特任教授は、国際機関のプロジェクト調整員や企画調査員として複数の国での活動経験があります。これらの国々の一般的な開発課題をいくつか挙げてください。その上で、高原特任教授が専門とする分野が、これらの開発課題の解決にどのように貢献できる可能性があるか、具体例を交えて説明してください。"
response_off = generate_output_qwen3(query, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF with RAG: {response_off}")

Thinking OFF with RAG: 高原敏竜特任教授は、国際機関のプロジェクト調整員や企画調査員として、複数の国での活動経験を持つ専門家です。その経験を通じて、発展途上国が直面する一般的な開発課題には以下のようなものがあります。

### 一般的な開発課題の例：
1. **情報通信インフラの整備不足**：インターネットや通信ネットワークの普及率が低く、遠隔地や農村部では情報アクセスが困難。
2. **技術人材の育成不足**：ICTを活用した技術者やリーダーが不足しており、持続可能な技術革新が困難。
3. **産業技術の低さ**：製造業やサービス業における技術レベルが低く、競争力が不足している。
4. **教育・研修制度の不十分さ**：技術やビジネスに関する教育・研修が不十分で、人材育成が困難。
5. **社会課題の解決へのICT活用の不足**：災害対応、環境保護、医療支援など、社会的な課題に対してICTを活用した解決策が少ない。

---

### 高原特任教授の専門分野がこれらの課題にどう貢献できるか（具体例）

高原特任教授の専門分野は、「ICTを活用したビジネスの価値づくり」や「技術者・技術マネジャー・DXリーダー人材育成」などです。これらの分野は、発展途上国における技術革新と社会課題の解決に直接的に貢献できる可能性があります。

#### 例1：**情報通信インフラの整備不足への貢献**
高原特任教授は、ICTを活用したソフトウェア開発や自動設計CADシステムの研究開発に長けています。これにより、遠隔地や農村部でも使える、コスト効率の高いICTインフラやアプリケーションを開発・導入することが可能です。例えば、スマートフォンやクラウド技術を活用した遠隔教育プラットフォームを開発し、教育資源の格差を縮小するプロジェクトに携わることで、情報通信インフラの整備不足を補うことができます。

#### 例2：**技術人材の育成不足への貢献**
高原特任教授は、技術研修やリーダー研修の企画運営に精通しており、技術者や技術マネジャーの育成に貢献できます。例えば、ICTを活用した技術研修プログラムを設計し、地元の技術者や学生に指導することで、持続可能な技術人材の育成を促進できます。また、技術リーダーとしてのスキルを高める研修を実施し、地元の企業や団体に技術支援を行う